### for data visualize

https://batuhandaz.medium.com/decision-tree-algoritmas%C4%B1-karar-a%C4%9Fac%C4%B1-machine-learning-78d856b1f457
https://ece-akdagli.medium.com/makine-%C3%B6%C4%9Frenmesinde-decision-tree-42a86502ee75
https://erdincuzun.com/makine_ogrenmesi/decision-tree-karar-agaci-id3-algoritmasi-classification-siniflama/

### Import Librarires And Dataset

In [1]:
import warnings
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

#some settings to show data
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

# target_url = ("http://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data")
target_url = ("datasets/abalone.data")
abalone_df = pd.read_csv(target_url)
headers = ['Sex', 'Length', 'Diameter', 'Height', 'Whole weight', 'Shucked weight', 'Viscera weight', 'Shell weight',
           'Rings']
abalone_df.columns = headers

### Analyze the Data

In [2]:
abalone_df.head()

,Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings
0,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
1,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
2,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
3,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7
4,I,0.425,0.300,0.095,0.3515,0.1410,0.0775,0.120,8


In [3]:
abalone_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4176 entries, 0 to 4175
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Sex             4176 non-null   object 
 1   Length          4176 non-null   float64
 2   Diameter        4176 non-null   float64
 3   Height          4176 non-null   float64
 4   Whole weight    4176 non-null   float64
 5   Shucked weight  4176 non-null   float64
 6   Viscera weight  4176 non-null   float64
 7   Shell weight    4176 non-null   float64
 8   Rings           4176 non-null   int64  
dtypes: float64(7), int64(1), object(1)
memory usage: 293.8+ KB


**Get target value**

In [4]:
# If you want the target values to be categorical rather than numeric, this process should be applied.

# for ix in abalone_df.index:
#     row = abalone_df.loc[ix]
#     if row["Rings"] <= 8:
#         abalone_df.loc[ix, 'Rings'] = 'Young'
#     elif row["Rings"] >= 11:
#         abalone_df.loc[ix, 'Rings'] = 'Old'
#     elif row["Rings"] >=9 & row["Rings"] <= 10:
#         abalone_df.loc[ix, 'Rings'] = 'Medium'

In [5]:
def is_numeric_value(x):
    return type(x) == int or type(x) == float

In [6]:
class MyQuestioner:
    def __init__(self, column, value):
        self.column = column
        self.value = value

    def __repr__(self):
        status = "=="
        if is_numeric_value(self.value):
            status = ">="
        return f"Is {headers[self.column]} {status} {self.value}"

    def compare(self, compared):
        val = compared[self.column]
        if is_numeric_value(val):
            return val >= self.value
        else:
            return val == self.value

In [7]:
def partitioner(rows, myQuestion):
    true_rows, false_rows = [], []
    for row in rows:
        if myQuestion.compare(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [8]:
def class_counts(rows):
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        label = row[-1]  # in our dataset format, the label is always the last column
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [9]:
def gini_impurity(rows):  # There are some ways like entropy, but I use gini impurity : 
    #  https://medium.com/machine-learning-t%C3%BCrkiye/karar-agaclari-algoritmasi-b823c23997d0#:~:text=Gini%20impurity%20nedir,%C3%B6l%C3%A7mek%20i%C3%A7in%20kullan%C4%B1labilir.

    impurityValue = 1
    counts = class_counts(rows)

    for label in counts:
        probability_of_label = counts[label] / float(len(rows))
        impurityValue -= probability_of_label ** 2

    return impurityValue

In [10]:
def information_gain(left, right, current_uncertainty):
    # Information Gain: The uncertainty of the starting node, minus the weighted impurity of two child nodes.

    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - (1 - p) * gini_impurity(right) - p * gini_impurity(left)

In [11]:
def find_the_best_split(rows):
    best_gain = 0  # best information gain
    best_question = None
    n_features = len(rows[0]) - 1  # column numbers
    current_uncertainty = gini_impurity(rows)

    for col in range(n_features):  # for each feature
        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value
            question = MyQuestioner(col, val)
            true_rows, false_rows = partitioner(rows, question)  # try partitioner

            if len(true_rows) == 0 or len(false_rows) == 0:  # if there is no dataset, skip
                continue

            gain = information_gain(true_rows, false_rows, current_uncertainty)

            if gain > best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [12]:
class Leaf:  # Leaf node is last of three, it has a value, has not any tree
    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [13]:
class Decision_Node:  # A Decision Node has two branch true or false
    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [14]:
def show_my_tree(node, spacing=""):
    if isinstance(node, Leaf):
        print(spacing + "Predict", predict_one(node.predictions))
        return

    print(spacing + str(node.question))

    print(spacing + '--> False/Left:')
    show_my_tree(node.false_branch, spacing + "  ")

    print(spacing + '--> True/Right:')
    show_my_tree(node.true_branch, spacing + "  ")

In [15]:
def classify(row, node):
    if isinstance(node, Leaf):
        return node.predictions

    if node.question.compare(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [16]:
def predict_one(counts):
    return max(counts, key=counts.get)

In [17]:
 def predict_all(X, tree):
    preditions = []
    # preditions = [max(classify(row, tree), key=classify(row, tree).get) for row in X]

    for row in X:
        cl = classify(row, tree)
        preditions.append(max(cl, key=cl.get))

    return preditions

In [18]:
def print_leaf(counts):
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [19]:
def builder(rows, max_depth, attribute_types, curr_depth=0):
    if (curr_depth <= max_depth):
        gain, question = find_the_best_split(rows)

        if gain == 0:
            return Leaf(rows)

        true_rows, false_rows = partitioner(rows, question)
        true_branch = builder(true_rows, max_depth, attribute_types, curr_depth + 1)  # Recursive call
        false_branch = builder(false_rows, max_depth, attribute_types, curr_depth + 1)  # Recursive call
        return Decision_Node(question, true_branch, false_branch)

    return Leaf(rows)

In [20]:
def build_dt(X, y, attribute_types, options):
    rows = np.concatenate((X, y), axis=1).tolist()
    return builder(rows, options["max_depth"], attribute_types)

In [21]:
X = abalone_df.iloc[:, :-1].values
y = abalone_df.iloc[:, -1].values.reshape(-1, 1)

options = {"max_depth": 2}
attribute_types = abalone_df.dtypes.apply(str).tolist()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=41)

my_tree = build_dt(X_train, y_train, attribute_types, options)

In [22]:
show_my_tree(my_tree)

Is Shell weight >= 0.145
--> False/Left:
  Is Diameter >= 0.225
  --> False/Left:
    Is Shell weight >= 0.022
    --> False/Left:
      Predict 4
    --> True/Right:
      Predict 5
  --> True/Right:
    Is Sex == I
    --> False/Left:
      Predict 9
    --> True/Right:
      Predict 7
--> True/Right:
  Is Shell weight >= 0.2575
  --> False/Left:
    Is Shucked weight >= 0.4315
    --> False/Left:
      Predict 9
    --> True/Right:
      Predict 9
  --> True/Right:
    Is Shell weight >= 0.3945
    --> False/Left:
      Predict 10
    --> True/Right:
      Predict 11


In [23]:
from sklearn.metrics import accuracy_score
y_pred = predict_all(X_test, my_tree)
accuracy_score(y_test, y_pred)

0.24760765550239233